# Roche SEIRD Model First Example

In this notebook we present how to use the `epimodels` module to set up an instantiation of the model built by F. Hoffmann-La Roche Ltd, using some toy data.

*The Roche model is built by F. Hoffmann-La Roche Ltd.*

In [1]:
# Load necessary libraries
import numpy as np
import epimodels as em
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the PHE Model

In [2]:
# Populate the model
regions = ['London', 'Cornwall']
age_groups = ['0-10', '10-25']

# Initial state of the system
contact_data_matrix_0 = np.array([[10, 5.2], [0, 3]])
contact_data_matrix_1 = np.array([[1, 0], [0, 3]])

region_data_matrix_0_0 = np.array([[1, 10], [1, 6]])
region_data_matrix_0_1 = np.array([[0.5, 3], [0.3, 3]])
region_data_matrix_1_0 = np.array([[0.85, 1], [0.9, 6]])
region_data_matrix_1_1 = np.array([[0.5, 0.2], [0.29, 4.6]])

contacts_0 = em.ContactMatrix(age_groups, contact_data_matrix_0)
contacts_1 = em.ContactMatrix(age_groups, contact_data_matrix_1)
regional_0_0 = em.RegionMatrix(
    regions[0], age_groups, region_data_matrix_0_0)
regional_0_1 = em.RegionMatrix(
    regions[1], age_groups, region_data_matrix_0_1)
regional_1_0 = em.RegionMatrix(
    regions[0], age_groups, region_data_matrix_1_0)
regional_1_1 = em.RegionMatrix(
    regions[1], age_groups, region_data_matrix_1_1)

# Matrices contact
matrices_contact = [contacts_0, contacts_1]
time_changes_contact = [1, 14]
matrices_region = [
    [regional_0_0, regional_0_1],
    [regional_1_0, regional_1_1]]
time_changes_region = [1, 14]

# NPIs data
max_levels_npi = [3, 3, 2, 4, 2, 3, 2, 4, 2]
targeted_npi = [True, True, True, True, True, True, True, False, True]
general_npi = [True, False, True, True, False, False, False, False, False]
reg_levels_npi = [
    [[0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 3, 2, 4, 2, 3, 2, 4, 2]],
    [[0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 3, 2, 4, 2, 3, 2, 4, 2]]]
time_changes_npi = [1, 14]

### Set the parameters and initial conditions of the model and bundle everything together

In [3]:
# Instantiate model
model = em.RocheSEIRModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)
model.set_age_groups(age_groups)
model.read_contact_data(matrices_contact, time_changes_contact)
model.read_regional_data(matrices_region, time_changes_region)
model.read_npis_data(max_levels_npi, targeted_npi, general_npi, reg_levels_npi, time_changes_npi)

# Initial number of susceptibles
susceptibles = [[1500, 600], [700, 400]]

# Initial number of infectives
infectives_pre = [[40, 20], [50, 32]]
infectives_pre_ss = [[2, 3], [10, 0]]
infectives_asym = [[0, 10], [0, 2]]
infectives_asym_ss = [[1, 1], [1, 0]]
infectives_sym = [[10, 20], [20, 32]]
infectives_sym_ss = [[4, 5], [1, 2]]

# Average times in compartments
k = 3.43
kS = 2.57
kQ = 1
kR = 9 * np.ones(len(age_groups))
kRI = 10 * np.ones(len(age_groups))

# Proportion of asymptomatic, super-spreader and dead cases
Pa = 0.658 * np.ones(len(age_groups))
Pss = 0.0955
Pd = 0.05 * np.ones(len(age_groups))

# Transmission parameters
beta_min = 0.228,
beta_max = 0.927
bss = 3.11
gamma = 1
s50 = 35.3

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, 20.5, 0.5).tolist()

### Simulate for one of the regions: **London**

In [4]:
# Simulate for all the regions
outputs = []

for r, reg in enumerate(regions):
    # List of initial conditions and parameters that characterise the model
    parameters = [
        r + 1, susceptibles, [[0, 0], [0, 0]], infectives_pre, 
        infectives_asym, infectives_sym, infectives_pre_ss,
        infectives_asym_ss, infectives_sym_ss, [[0, 0], [0, 0]],
        [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]],
        k, kS, kQ, kR, kRI, Pa, Pss, Pd,
        beta_min, beta_max, bss, gamma, s50, 'RK45']

    # Simulate using the ODE solver
    outputs.append(model.simulate(list(deepflatten(parameters, ignore=str)), times))

## Plot the comparments of the two methods against each other
### Setup ``plotly`` and default settings for plotting

In [5]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']

### Plot the comparments of the two methods against each other

In [6]:
# Trace names - represent the solver used for the simulation
trace_name = ['region {}'.format(r) for r in regions]

# Compartment list - type and age
comparments = []
for n in model.output_names():
    comparments.append('{}'.format(n))

# Plot for each comparment
for c, comparment in enumerate(comparments):
    fig = go.Figure()
    fig = make_subplots(rows=int(np.ceil(len(age_groups)/2)), cols=2, subplot_titles=tuple('ages {}'.format(a) for a in age_groups))
    # Plot (line plot for each solver method for each age)
    for a, age in enumerate(age_groups):
        if a != 0:
            for o, out in enumerate(outputs):
                fig.add_trace(
                    go.Scatter(
                        y=out[:, c*len(age_groups)+a],
                        x=times,
                        mode='lines',
                        name=trace_name[o],
                        line_color=colours[o],
                        showlegend=False
                    ),
                    row= int(np.floor(a / 2)) + 1,
                    col= a % 2 + 1
                )
        
        else:
            for o, out in enumerate(outputs):
                fig.add_trace(
                    go.Scatter(
                        y=out[:, c*len(age_groups)+a],
                        x=times,
                        mode='lines',
                        name=trace_name[o],
                        line_color=colours[o]
                    ),
                    row= int(np.floor(a / 2)) + 1,
                    col= a % 2 + 1
                )

    # Add axis labels
    fig.update_layout(
        boxmode='group',
        title=comparment, 
        width=800,
        plot_bgcolor='white',
        xaxis=dict(linecolor='black'),
        yaxis=dict(linecolor='black'),
        xaxis2=dict(linecolor='black'),
        yaxis2=dict(linecolor='black'),
        xaxis3=dict(linecolor='black'),
        yaxis3=dict(linecolor='black'),
        xaxis4=dict(linecolor='black'),
        yaxis4=dict(linecolor='black'),
        xaxis5=dict(linecolor='black'),
        yaxis5=dict(linecolor='black'),
        xaxis6=dict(linecolor='black'),
        yaxis6=dict(linecolor='black'),
        xaxis7=dict(linecolor='black'),
        yaxis7=dict(linecolor='black'),
        xaxis8=dict(linecolor='black'),
        yaxis8=dict(linecolor='black'),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ))

    fig.show()

### Plot the super-spreader symptomatic infectious comparment for one region (London)

In [11]:
# Trace names - represent the solver used for the simulation
trace_name = ['ages {}'.format(a) for a in age_groups]

# Compartment list - type and age
fig = go.Figure()

# Plot (line plot for each method of simulation)
for a in range(len(age_groups)):
    fig.add_trace(
        go.Scatter(
            y=outputs[0][:, 8 * len(age_groups) + a],
            x=times,
            mode='lines',
            name=trace_name[a],
            line_color=colours[a]
        )
    )

# Add axis labels
fig.update_layout(
    boxmode='group',
    title='Iss in region {}'. format(regions[0]),
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

fig.write_image('images/Toy.pdf')
fig.show()